In [1]:
import numpy as np
from environment.gobang import WrapperGoBang
from agent import Agent
import random
import os
from loguru import logger

In [2]:
class GoBangRandomAgent(Agent):
    def step(self, obs, explore=True, legal_actions=None):
        return {'action': random.choice(legal_actions)}

In [3]:
def do_exp(agent, gobang_n, epochs, test_n, test_freq, result_path):
    random_agent = GoBangRandomAgent(None)
    loss_episode = []
    reward_episode = []
    win_ratio = []
    best_win_ratio = -np.inf
    env = WrapperGoBang(gobang_n)
    for i in range(epochs):
        loss_list, reward_sum = agent.train(env)
        logger.info(f'Episode {i}/{epochs}, average loss: {np.mean(loss_list) if loss_list else -1}, reward sum: {reward_sum}')
        if loss_list:
            loss_episode.append(np.array([i, np.mean(loss_list)]))
            reward_episode.append(np.array([i, reward_sum]))
        
        if i % test_freq == 0:
            win = 0
            for _ in range(test_n):
                win += agent.test(env, random_agent, display=False)
            logger.info(f'Episode {i}/{epochs}, win {win}/{test_n}')
            ratio = 1.0 * win / test_n
            win_ratio.append(np.array([i, ratio]))
            if ratio > best_win_ratio:
                agent.save()
    np.save(os.path.join(result_path, 'loss.npy'), np.array(loss_episode))
    np.save(os.path.join(result_path, 'reward.npy'), np.array(reward_episode))
    np.save(os.path.join(result_path, 'ratio.npy'), np.array(win_ratio))    
    

In [4]:
from agent import DQNAgent
from agent.model import ConvQModel
from agent.configuration import DQNConfig
from agent.exploration import LinearExploration

In [5]:
eval_model = ConvQModel(
    in_channels=2,
    in_h=7,
    in_w=7,
    hidden_channels=32,
    output_dim=49,
    dueling=True
)
target_model = ConvQModel(
    in_channels=2,
    in_h=7,
    in_w=7,
    hidden_channels=32,
    output_dim=49,
    dueling=True
)
exploration = LinearExploration(
    init_epsilon=1.0,
    min_epsilon=0.1,
    epsilon_decay=0.995
)
config = DQNConfig(
    eval_model=eval_model,
    target_model=target_model,
    ckpt_path='./dueling_ddqn_gobang_n7.ckpt',
    batch_size=32,
    lr=0.001,
    gamma=0.99,
    target_replace_frequency=100,
    capacity=5000,
    max_grad_norm=1.0,
    exploration=exploration,
    ddqn=True
)
agent = DQNAgent(config)
do_exp(agent, 7, 5000, 50, 50, './experiment/dueling_ddqn_gobang_n7')

2023-05-05 05:14:08.801 | INFO     | __main__:do_exp:10 - Episode 0/5000, average loss: 0.1740681612632784, reward sum: -38
2023-05-05 05:14:17.715 | INFO     | __main__:do_exp:19 - Episode 0/5000, win 28/50
2023-05-05 05:14:18.523 | INFO     | __main__:do_exp:10 - Episode 1/5000, average loss: 0.02733665597784495, reward sum: -45
2023-05-05 05:14:19.356 | INFO     | __main__:do_exp:10 - Episode 2/5000, average loss: 0.009320676441355, reward sum: -35
2023-05-05 05:14:20.107 | INFO     | __main__:do_exp:10 - Episode 3/5000, average loss: 0.0796927089556696, reward sum: -32
2023-05-05 05:14:20.987 | INFO     | __main__:do_exp:10 - Episode 4/5000, average loss: 0.021838339332665886, reward sum: -44
2023-05-05 05:14:21.828 | INFO     | __main__:do_exp:10 - Episode 5/5000, average loss: 0.0405045904467013, reward sum: -37
2023-05-05 05:14:22.436 | INFO     | __main__:do_exp:10 - Episode 6/5000, average loss: 0.0500333524302163, reward sum: -27
2023-05-05 05:14:23.154 | INFO     | __main__:

In [6]:
eval_model = ConvQModel(
    in_channels=2,
    in_h=11,
    in_w=11,
    hidden_channels=32,
    output_dim=121,
    dueling=True
)
target_model = ConvQModel(
    in_channels=2,
    in_h=11,
    in_w=11,
    hidden_channels=32,
    output_dim=121,
    dueling=True
)
exploration = LinearExploration(
    init_epsilon=1.0,
    min_epsilon=0.1,
    epsilon_decay=0.995
)
config = DQNConfig(
    eval_model=eval_model,
    target_model=target_model,
    ckpt_path='./dueling_ddqn_gobang_n11.ckpt',
    batch_size=32,
    lr=0.001,
    gamma=0.99,
    target_replace_frequency=100,
    capacity=5000,
    max_grad_norm=1.0,
    exploration=exploration,
    ddqn=True
)
agent = DQNAgent(config)
do_exp(agent, 11, 5000, 50, 50, './experiment/dueling_ddqn_gobang_n11')

2023-05-05 06:55:34.005 | INFO     | __main__:do_exp:10 - Episode 0/5000, average loss: 0.042493659634572294, reward sum: -72
2023-05-05 06:56:29.666 | INFO     | __main__:do_exp:19 - Episode 0/5000, win 8/50
2023-05-05 06:56:31.925 | INFO     | __main__:do_exp:10 - Episode 1/5000, average loss: 0.04955223226549396, reward sum: -77
2023-05-05 06:56:35.171 | INFO     | __main__:do_exp:10 - Episode 2/5000, average loss: 0.03301290614298235, reward sum: -82
2023-05-05 06:56:37.165 | INFO     | __main__:do_exp:10 - Episode 3/5000, average loss: 0.04383213837541784, reward sum: -41
2023-05-05 06:56:40.465 | INFO     | __main__:do_exp:10 - Episode 4/5000, average loss: 0.02942935816913336, reward sum: -59
2023-05-05 06:56:44.126 | INFO     | __main__:do_exp:10 - Episode 5/5000, average loss: 0.028646745662795504, reward sum: -74
2023-05-05 06:56:48.200 | INFO     | __main__:do_exp:10 - Episode 6/5000, average loss: 0.032296849702728204, reward sum: -89
2023-05-05 06:56:50.929 | INFO     | __